In [ ]:
    def fit_poisson(alpha, train_x, train_y, val_x):
        val_pred = []
        train_x =  np.log(train_x + config['setup']['log_eps'])
        val_x =  np.log(val_x + config['setup']['log_eps'])
        for chan in range(train_y.shape[1]):
            pr = PoissonRegressor(alpha=alpha, max_iter=500)
            pr.fit(train_x, train_y[:, chan])
            while pr.n_iter_ == pr.max_iter and pr.max_iter < 10000:
                print(f"didn't converge - retraining {chan} with max_iter={pr.max_iter * 5}")
                oldmax = pr.max_iter
                del pr
                pr = PoissonRegressor(alpha=alpha, max_iter=oldmax * 5)
                pr.fit(train_x, train_y[:, chan])
            val_pred.append(pr.predict(val_x))
        val_rates_s = np.vstack(val_pred).T
        return np.clip(val_rates_s, 1e-9, 1e20)

    kf = KFold(n_splits=5)

    stds = [30,  40, 50, 60, ]
    # stds = [10, 20, 30,  40, 50, 60, 70, 80,  90,  110, 130, 150]
    
    filters = []
    for std in stds:
        kern_sd = int(round(std / 10))
        window = signal.gaussian(kern_sd * 6, kern_sd, sym=True)
        window /= np.sum(window)
        filters.append(lambda x: np.convolve(x, window, 'same'))

    train_spikes_heldin = h5dict['all_train_spikes_heldin'][:, -1, :]
    train_spikes_heldout = h5dict['all_train_spikes_heldout'][:, -1, :]
    test_spikes_heldin = h5dict['test_spikes_heldin'][:, -1, :]
    test_spikes_heldout = h5dict['test_spikes_heldout'][:, -1, :]

    for std, filter in zip(stds, filters):
        for alpha in np.logspace(-3, 0, 4):
            split = []
            split_test = []
            for train_index, val_index in kf.split(train_spikes_heldin):
                heldin_smth_spikes= np.apply_along_axis(filter, 0, train_spikes_heldin)
                train_hi = heldin_smth_spikes[train_index]
                val_hi = heldin_smth_spikes[val_index]
                train_ho = train_spikes_heldout[train_index]
                val_ho = train_spikes_heldout[val_index]
                val_rates = fit_poisson(alpha, train_hi, train_ho, val_hi)
                split.append(bits_per_spike(np.expand_dims(val_rates, 1), np.expand_dims(val_ho, 1)))
            print('alpha:',alpha,'std:',std)
            print('val mean:', np.mean(np.array(split)))
            train_hi= np.apply_along_axis(filter, 0, train_spikes_heldin)
            train_ho = train_spikes_heldout
            test_hi= np.apply_along_axis(filter, 0, test_spikes_heldin)
            test_ho = test_spikes_heldout
            val_rates = fit_poisson(alpha, train_hi, train_ho, test_hi)
            print('test:', bits_per_spike(np.expand_dims(val_rates, 1), np.expand_dims(test_ho, 1)))